In [9]:
import cv2
import numpy as np

# Step 1: Load the two images
# image2 = cv2.imread("Bedroom/Holiday Inn - Bedroom Adjustment 2.jpg")
# image1 = cv2.imread("Bedroom/Holiday Inn - Bedroom Baseline Pic.jpg")
image2 = cv2.imread("Bathroom/Holiday Inn - Bathroom Adjustments 3.jpg")
image1 = cv2.imread("Bathroom/Holiday Inn - Bathroom Baseline Pic.jpg")

# Step 2: Convert images to grayscale for optical flow
gray_image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
gray_image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

# Step 3: Perform optical flow using Lucas-Kanade algorithm
lk_params = dict(winSize=(15, 15), maxLevel=5, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
points1 = cv2.goodFeaturesToTrack(gray_image1, mask=None, maxCorners=200, qualityLevel=0.01, minDistance=10)

if points1 is not None:
    points2, status, _ = cv2.calcOpticalFlowPyrLK(gray_image1, gray_image2, points1, None, **lk_params)

    # Filter points and create corresponding keypoints
    keypoints1 = [cv2.KeyPoint(x[0][0], x[0][1], 10) for x in points1]
    keypoints2 = [cv2.KeyPoint(x[0][0], x[0][1], 10) for x in points2]

    # Convert keypoints to numpy arrays
    keypoints1 = np.array([kp.pt for kp in keypoints1], dtype=np.float32).reshape(-1, 1, 2)
    keypoints2 = np.array([kp.pt for kp in keypoints2], dtype=np.float32).reshape(-1, 1, 2)

    # Filter keypoints based on status
    keypoints1 = keypoints1[status == 1]
    keypoints2 = keypoints2[status == 1]

    # Step 4: Estimate the transformation matrix using RANSAC
    M, _ = cv2.findHomography(keypoints1, keypoints2, cv2.RANSAC, 5.0)
    aligned_image = cv2.warpPerspective(image1, M, (image2.shape[1], image2.shape[0]))

    # Rest of the code remains the same...


else:
    print("No points found for optical flow.")

# Step 4: Perform histogram matching to adjust for differences in lighting and intensity
aligned_image = cv2.cvtColor(aligned_image, cv2.COLOR_BGR2LAB)
image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2LAB)

aligned_image[:, :, 0] = cv2.createCLAHE(clipLimit=1.5, tileGridSize=(8, 8)).apply(aligned_image[:, :, 0])
image2[:, :, 0] = cv2.createCLAHE(clipLimit=1.5, tileGridSize=(8, 8)).apply(image2[:, :, 0])

aligned_image = cv2.cvtColor(aligned_image, cv2.COLOR_LAB2BGR)
image2 = cv2.cvtColor(image2, cv2.COLOR_LAB2BGR)

# Rest of the code remains the same...


# Step 5: Calculate the absolute difference between the aligned image and image 2
diff = cv2.absdiff(aligned_image, image2)
gray_diff = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)

# Step 6: Denoise the difference image
denoised_diff = cv2.fastNlMeansDenoising(gray_diff, h=200, templateWindowSize=7, searchWindowSize=11)

# Step 7: Threshold the denoised difference image to identify significant differences
threshold = 65  # Adjust this threshold as needed
_, thresh = cv2.threshold(denoised_diff, threshold, 255, cv2.THRESH_BINARY)

# Step 8: Find contours of the significant differences
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Merge overlapping bounding boxes and filter small boxes
contours_merged = []
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    if w * h > 100:  # Adjust this size threshold as needed
        contours_merged.append((x, y, x + w, y + h))

# Sort the merged contours by their y-coordinate to ensure non-overlapping boxes
contours_merged = sorted(contours_merged, key=lambda x: x[1])

# Create a separate list to store the non-overlapping contours
non_overlapping_contours = []

# Iterate over the contours_merged list
for x1, y1, x2, y2 in contours_merged:
    overlapping = False
    for x3, y3, x4, y4 in non_overlapping_contours:
        # Check if boxes overlap
        if x3 < x2 and x4 > x1 and y3 < y2 and y4 > y1:
            overlapping = False
            break
    # Add the non-overlapping box to the list
    if not overlapping:
        non_overlapping_contours.append((x1, y1, x2, y2))

# Draw non-overlapping bounding boxes on image 2
highlighted_image = image2.copy()
for x1, y1, x2, y2 in non_overlapping_contours:
    cv2.rectangle(highlighted_image, (x1, y1), (x2, y2), (0, 0, 255), 1)

# Step 9: Display the side-by-side comparison of the images
combined_image = np.concatenate((image1, highlighted_image), axis=1)
cv2.imshow("Image Comparison", combined_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
